# Применение генетического алгоритма на данных Criteo. Данный код относится к разделу 1.5 отчета о НИР.

In [1]:
import pandas as pd
import numpy as np
import random
import math
import joblib
import scipy
from tqdm.notebook import tqdm
from scipy.stats import chi2_contingency, kstest, ks_2samp, ttest_ind
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import normalize

In [2]:
## Для скачивания датасета раскомментировать следующий код
#!wget http://go.criteo.net/criteo-research-uplift-v2.1.csv.gz
#!gzip -d ./criteo-research-uplift-v2.1.csv.gz

В случае, если у вас не работает утилита wget - датасет нужно скачать вручную по ссылке выше и распаковать из архива в .csv файл с названием 'criteo-research-uplift-v2.1.csv'

In [3]:
inp_df = pd.read_csv('criteo-research-uplift-v2.1.csv')
features = ['f{}'.format(n) for n in range(12)]
inp_df[features] = normalize(inp_df[features], axis=0, norm='l2')

X = features
y = 'visit'
t = 'treatment'

inp_df.shape

(13979592, 16)

In [4]:
cat_features = ['f1', 'f3', 'f4', 'f5', 'f6', 'f8', 'f9', 'f11']
noncat_features = list(inp_df.columns.difference(cat_features + [y,t,'conversion', 'exposure']))

df = inp_df
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
0,0.000166,0.000267,0.000284,0.000285,0.000266,0.000272,0.000013,0.000247,0.000269,0.000202,0.000266,-0.000262,1,0,0,0
1,0.000166,0.000267,0.000285,0.000285,0.000266,0.000272,0.000013,0.000247,0.000269,0.000202,0.000266,-0.000262,1,0,0,0
2,0.000166,0.000267,0.000284,0.000285,0.000266,0.000272,0.000013,0.000247,0.000269,0.000202,0.000266,-0.000262,1,0,0,0
3,0.000166,0.000267,0.000285,0.000285,0.000266,0.000272,0.000013,0.000247,0.000269,0.000202,0.000266,-0.000262,1,0,0,0
4,0.000166,0.000267,0.000286,0.000285,0.000266,0.000272,0.000013,0.000247,0.000269,0.000202,0.000266,-0.000262,1,0,0,0


In [5]:
df['ID'] = df.index.astype(int)
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure,ID
0,0.000166,0.000267,0.000284,0.000285,0.000266,0.000272,0.000013,0.000247,0.000269,0.000202,0.000266,-0.000262,1,0,0,0,0
1,0.000166,0.000267,0.000285,0.000285,0.000266,0.000272,0.000013,0.000247,0.000269,0.000202,0.000266,-0.000262,1,0,0,0,1
2,0.000166,0.000267,0.000284,0.000285,0.000266,0.000272,0.000013,0.000247,0.000269,0.000202,0.000266,-0.000262,1,0,0,0,2
3,0.000166,0.000267,0.000285,0.000285,0.000266,0.000272,0.000013,0.000247,0.000269,0.000202,0.000266,-0.000262,1,0,0,0,3
4,0.000166,0.000267,0.000286,0.000285,0.000266,0.000272,0.000013,0.000247,0.000269,0.000202,0.000266,-0.000262,1,0,0,0,4


In [6]:
df.columns

Index(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'treatment', 'conversion', 'visit', 'exposure', 'ID'],
      dtype='object')

In [7]:
# Признаки, по которым будет производиться стратификация
strat_features = ["f0", "f2", "f8", "f9", "f1", "f5", "f11"]

In [8]:
cat_features = ["f8", "f9", "f1", "f5", "f11"]
num_features = ["f0", "f2"]

# Код стратификации Индустриального Партнера

In [9]:
def get_base_stratification(df, df_aa, p_key:str, part):  
    """
    За основу взяли функцию true_strat. Данная функция возвращает разбиение данных на страты.
    """ 
    ERR=0
    flag_group = 0
    mes = ""
    
        
    if(p_key not in df.columns):
        ERR = 1
        mes = "ОШИБКА: В данных не найден столбец содержащий объекты деления!"
        
    elif(len(df.columns)<2):  
        ERR = 2
        mes = "ОШИБКА: Не найдены столбцы с критериями однородности!"
        
    elif(df[p_key].duplicated().any() == True):   
        ERR = 3
        mes = "ОШИБКА: В ключевом столбце найдены дубликаты! Значения должны быть уникальными." 
        

    if(ERR > 0): # Если есть ошибки, то запуск деления не возможен    
        
        print(mes)
        
        return ERR
        
    else:
        
        df = sorting_features(df_res = df, p_key = p_key) # сортируем фичи по типу, чистим данные

        form_strat_ = form_strat(df, part) # формируем страты
        return form_strat_

## Формирование страт

In [10]:
# Формирование страт
"""
Функция скрипта:  формирование страт.

Входящие параметры: 
df – таблица с данными по участникам
part - количество групп, формат – целые числа, в MVP0 значение всегда 2
"""
    
def form_strat(df, part):
    
    features = []
    strat_feature = []
    
    features_strat = ['f0 : ks', 'f2 : ks', 'f8 : ks','f9 : ks', 'f1 : ks', 'f5 : ks', 'f11 : ks']
    
    kombo={}
    strat_kombo = {}
    strat = {}
    
    for i in df.columns:  
        if(i in features_strat):
            if('ks' in i):
                df[i.replace('ks', 'interval')] = interval_combo(df[i]) # формируем интервалы из числовых фичей
                features.append(i.replace('ks', 'interval'))
            else:
                features.append(i)   
# группируем данные
    for i in features:          
        strat_feature.append({i:df.groupby(i).groups})


    for i in strat_feature:
        for k in i:        
        
            iloc={}
        
            kombo[k] = iloc
        
            for j in i[k]:        
            
                iloc[k+"_"+str(j)] = list(i[k][j])


    for q in range(len(features)):
        
        if(q == 0):
            col_unic = kombo[features[q]]
        else: 
            col_unic = strat
    
        if(q+1<len(features)):
            strat = {}
    
            for e in col_unic:
            
                for r in kombo[features[q+1]]:
                    cells_strat = list(set(col_unic[e]) & set(kombo[features[q+1]][r]))
                    

                    
                    #print("Последний столбец:",features[q])
                    if(q == len(features)-2):
                        #print("Последний столбец:",features[q])
                        if(len(cells_strat)>0): # если страта не пустая \ больше или равна кол-ву групп, записываем участников
                            
                            if(len(cells_strat)<part):
                                if("one_strat" not in strat):
                                    strat["one_strat"] = cells_strat
                                else:    
                                    strat["one_strat"].append(cells_strat[0])
                            #elif(len(cells_strat)==3 and part == 2):
                               # strat[(str(e)+"__"+str(r)).replace(" : xi2", "")] = cells_strat[:2]
                               # if("one_strat" not in strat):
                                    #strat["one_strat"] = cells_strat[2]
                                #else:    
                                    #strat["one_strat"].append(cells_strat[2])
                                #print("\n","Три участника страты:", strat[(str(e)+"__"+str(r)).replace(" : xi2", "")], cells_strat[:2])
                      #print("Нормальная страта:", (str(e)+"__"+str(r)).replace(" : xi2", ""))
                            else:
                                strat[(str(e)+"__"+str(r)).replace(" : xi2", "")] = cells_strat
                        
                    else:    
                        
                        strat[(str(e)+"__"+str(r)).replace(" : xi2", "")] = cells_strat
                        
    if(len(strat)==0):
        strat={'one_strat': list(df.index)}
  
    return strat  



## Отбор числа участников из страты

#### Пропорциональное размещение

Считаем, что сумма числа наблюдений, попавших во все страты, совпадает с числом наблюдений в генеральной совокупности (ГС).

Далее фиксируется объем выборки $n$, необходимый для проведения теста на заданном уровне значимости (например, в соответствии с формулой минимального размера выборки). Далее число наблюдений $n_h$, выбираемых из страты номер $h$, вычисляется в соответствии с формулами из раздела Отчета о НИР 1.2.

Исключение из правил:

* При округлении количества наблюдений, генерируемых из страты, общее число участников теста может получиться меньше, чем заявленное изначально число $n$. Возможное решение - просто округлять размер подвыборок в большую сторону.

### Формирование интервалов для числовых фичей

### Метод процентилей

In [11]:
"""
Функция скрипта:  формирование интервалов в числовой фиче.

Входные параметры скрипта:
s – отдельная фича, столбец

На выходе скрипта:

s_percentile, столбец содержащий интервалы для фичи, которая загружалась при вызове скрипта. 

"""

def interval_combo(s):
    
    s_percentile = pd.Series(list(s.values), index=list(s.index)) 
    
    p = [0.5, 0.25, 0.05] 
    
    for i in p:
        s_percentile[list(s[(s<s.quantile(q=i))].index)] = i
        s_percentile[list(s[(s>=s.quantile(q=1.0-i))].index)] = i
        
    s_percentile[list(s[(s == 0)].index)] = 0 # ноль выделяем в отдельный интервал 
    s_percentile[list(s[(s < 0)].index)] = -1 # минусовые значения выделяем в отдельный интервал - т.к. убытки
     
            
    return s_percentile


## Сводный скрипт стратификации

In [12]:
"""
Функция скрипта:  сортировка фичей по критериям + чистка данных в фичах.

Входящие параметры: 
df_res – таблица с данными участников
p_key -  p_key – название ключевого столбца (содержит объекты деления), формат – строковый, 
значение берется из входного параметра p_key скрипта, который вызывает скрипт sorting_features

flag_aa = False - флаг АА-теста, выбор будет ли проводиться АА-тест или стандартная проверка на однородность.

"""

def sorting_features(df_res, p_key:str, flag_aa = False): # сортировка фичей по критериям + чистка данных в фичах
    
    features_ks = ['f0', 'f2', 'f8', 'f9', 'f1', 'f5','f11']
    features_xi2 = []
    features_case = []
    

    list_kriterii=[]

    for i in df_res.columns:  
        
        #print(i,'тип фичи', df_res[i].dtype)
        
        if((i == p_key) or (i == 'group_num') or ("ks" in i) or ("xi2" in i) or ("interval" in i) or ("tt" in i)):
            list_kriterii.append(i)
            
        else:
                    
            if(i in features_xi2):
                #print("Хи2 1", i)
                list_kriterii.append(i+" : xi2")
                
            elif(i in features_ks and (df_res[i].dtype == np.float64 or df_res[i].dtype == np.int64) and flag_aa == False):
                #print("ks", i)
                list_kriterii.append(i+" : ks")
            
            elif(df_res[i].dtype != np.float64 and df_res[i].dtype != np.int64):
                #print("Нужна чистка", i)
                
                # чистка данных в  фиче
                df_res[i] = df_res[i].replace({'null':np.nan, 'Null':np.nan,'NULL':np.nan,'nan':np.nan,'Nan':np.nan,'NaN':np.nan,'NAN':np.nan,'-':np.nan, '':np.nan, ' ':np.nan})          
                
                try:
                    # перевод фичи в числовой формат
                    df_res = df_res.astype({i: "float64"})
                except:  
                    if(i in features_ks or i in features_case):
                        print("Ошибка при переводе в числовой формат фичи:", i, " Данные в фиче не относятся к числовым, требуется замена данных.")
                         
            # анализ данных в фиче, после чистки данных
            
            if(i +" : ks" not in list_kriterii and i+" : xi2" not in list_kriterii and i+" : tt" not in list_kriterii): 
                if(flag_aa == True and (df_res[i].dtype == np.float64 or df_res[i].dtype == np.int64)):
                    list_kriterii.append(i+" : tt")
                
                elif(df_res[i].nunique()>7 and (df_res[i].dtype == np.float64 or df_res[i].dtype == np.int64)):
                        #print("KS", i)                
                        list_kriterii.append(i+" : ks")
                    
                else:
                    #print("Хи2 2", i)
                    list_kriterii.append(i+" : xi2")
                    
           
                
    df_res.columns = list_kriterii                
    return df_res

### Проверка однородности - получаем p-value

In [13]:
# скрипт хи2

def ser_conjugacy(ser, index):
    zero=[]        
    index_plus = list(set(index) - set(ser.index.tolist()))
 
    for i in range (len(index_plus)):
        zero.append(0)
    
    return pd.concat([ser, pd.Series(zero, index=index_plus)]) 

"""
Функция скрипта: Построение таблицы сопряженности с учетом малонаполненных подгрупп, проведение стат. теста Хи2.

Входящие параметры: 
gr_A - данные участников по столбцу от одной группы
gr_B - данные участников по столбцу от другой группы


"""

def check_homo_cat_xi2_other(gr_A,gr_B): 
    nonhomo={}
    other={}
    count=0

    # строим таблицу сопряженности
    gr_B=gr_B.groupby(gr_B).count()
    gr_A=gr_A.groupby(gr_A).count()
    index_= pd.Series(gr_B.index.tolist()+gr_A.index.tolist())
    index_= index_.groupby(index_).count().index.tolist()
    
    gr_B = ser_conjugacy(gr_B, index_)
    gr_A = ser_conjugacy(gr_A, index_)    
   
    merge_data=pd.DataFrame()
    merge_data['id_x']=gr_A
    merge_data['id_y']=gr_B
    merge_data=merge_data.fillna(0)     # заполнение пропусков нулями
   
    # Обрабатывем малонаполненные группы
    count_sub_gr=(merge_data['id_x']+merge_data['id_y']).sort_values()    
    
    
    # если есть малонаполненные подгруппы, то объединяем их
    for i in count_sub_gr.index:   
           
        if((count_sub_gr.loc[i]<=15) or (count==1)):
            
            if(count==0):                                          
                other = {'id_x': merge_data['id_x'].loc[i],'id_y':merge_data['id_y'].loc[i]}   
                
            else:  
                other = {'id_x': merge_data['id_x'].loc[i]+other['id_x'],'id_y':merge_data['id_y'].loc[i]+other['id_y']}  
            
            count+=1
            merge_data.drop(labels = [i],axis = 0, inplace = True)                
            
            
    if(len(other)>0):                
            merge_data = pd.concat([merge_data, pd.DataFrame([other],index =['Прочее'])]) # , ignore_index=True
    #print("\n")  
    
          
    pvalue = chi2_contingency(merge_data[["id_x","id_y"]])[1]    
        
    #print('p-value:', round(pvalue,5))    
    if (str(pvalue)=='nan'):       
        
        pvalue = 0
        
    
    return pvalue


In [14]:
# Тест Колмогорова-Смирнова

"""
Функция скрипта: Проведение стат. теста Колмогорова-Смирнова.

Входящие параметры: 
gr_A - данные участников по столбцу от одной группы
gr_B - данные участников по столбцу от другой группы
alpha  –  уровень значимости, формат – дробное число, значение берется из входного параметра alpha  скрипта, который вызывает  скрипт ks_test

"""
def ks_test(gr_A,gr_B,alpha):
    ks_test=ks_2samp(gr_A,gr_B)[1]
    #if(ks_test<alpha):
        #print("Колмогоров-Смирнов, НЕ однородны: р=",ks_test)    
    #else:
        #print("Колмогоров-Смирнов, однородны: р=",ks_test) 
    
    return ks_test
    


In [15]:
"""
Функция скрипта: проверка однородности, формирования отчета по фичам
Входящие параметры: 

•	df_res – таблица сформированная скриптами random_strat или multy_group•	
•	alpha  –  уровень значимости, формат – дробное число, значение берется из входного параметра alpha  скрипта, который вызывает  скрипт checking_uniformity
•	p_group - название столбца с указанием принадлежности к группе таблицы df_res, формат - строковый, если скрипт вызывается скрипами random_strat или multy_group, значение «».
•	part -  количество групп, формат – целые числа, значение берется из входного параметра part скрипта, который вызывает скрипт checking_uniformity
"""

def checking_uniformity(df_res, p_group:str, part, alpha):
    flag_group=0      

    
    df_report = pd.DataFrame()    
    df_report_new  = pd.DataFrame()
    
    
    for i in df_res.columns:   
                        
        if('Unnamed' in i):                
            df_res.drop(i, axis=1, inplace=True)
            
        elif(('xi2' in i) or ('ks' in i) or ('tt' in i)):
                        
            
#####################################################################################################3                        
               
                        # проверка на две группы  
            if(part==2):
                #print("две группы")
                        
                gr_A=df_res[(df_res["group_num"] == 0 )][i] 
                gr_B=df_res[(df_res["group_num"] == 1 )][i]                         
                        
                    # если распределение биноминальное или категориальное - отправляем на хи2, кат - не более 7 уникальзых значений. 
                
                
                
                if('xi2' in i):
                            
                    #print("бином или категория", i, df_res[i].dtypes)  #фича на хи2                    
                    p_value = check_homo_cat_xi2_other(gr_A,gr_B)
                    #res=[i,p_value,alpha] 
                    
                        #break  
                            
                            
                elif('ks' in i):     
                    # числовые отправляются на критерий Колмогорова-Смирнова
                    #print("колмогоров", i, df_res[i].dtypes)  
                    
                    p_value = ks_test(gr_A,gr_B,alpha) 
                    #res=[i,p_value,alpha] 
                     # Заполняем данными по итерации датафрейм сплитовалки  
                elif('tt' in i):   
                    
                    p_value = t_test(gr_A,gr_B, alpha)
                    
                        #break #если однородность не найдена на одной из колонок, следующие не проверяются, переход к следующей итерации
            # если нет однородности
                
                if(p_value < alpha):                        
                    flag = 0  
                else:
                    flag = 1
                    
            df_report_new = pd.DataFrame({'features':i, 'p-value':p_value, 'alpha':alpha, 'flag':flag},index = [0])
        
            df_report = pd.concat([df_report, df_report_new], ignore_index=True)
                        
    
    if(df_report['flag'].nunique()==1 and df_report['flag'].unique()==1):
        flag_group = 1 
    #print("Флаг однородности:", flag_group)
    
    # flag_group = 1, значит однородно
    return flag_group, df_report 
 

### Отбор и деление

In [16]:
# деление на группы

"""
Функция скрипта: деление на группы 

Входящие параметры: 
df - данные по участникам страты, формат – таблица,
part - количество групп, формат – целые числа.

"""
def find_group(df,part:int):        
    
    df_new = pd.DataFrame()
    df_res = pd.DataFrame()
    
    if(len(df)>0):
        n_h = len(df)
        df.reset_index(drop=True, inplace=True) # сбрасываем индексы
        n_h_group = round(n_h/part)
        
        if(n_h_group > 0):
            for i in range(part):          
            
                if(i!=(part-1)): 
                
                                   
                        df_new=df.sample(n=n_h_group)             
                        df=df[~df.index.isin(df_new.index)]  
            
                else:    
                    df_new=df
            
        
                df_new['group_num']=i    
                df_res = pd.concat([df_res, df_new], ignore_index=True) #
        
        
    return df_res   

In [17]:
"""
Функция скрипта: При репрезентативном отборе пропорционально вычисляет размер страты для тестовой выборки.

Входящие параметры: 
df_strat – данные по участникам страты, формат – таблица
n_gs – количество участников в ГС, формат - целые числа
n – количество участников в выборке, формат - целые числа
part - количество групп, формат – целые числа, в MVP0 значение всегда 2.

"""

def select_prop(df_strat, n_gs:int, n:int, part:int):
    
    per_strat = len(df_strat)/n_gs # находим процент, занимаемый стратой в ГС
        
    
    n_h =  round(n*per_strat) 
    if(n_h > 0 and n_h < part):
        n_h = part # отрез участников для малых страт    
        
    if(part == 2 and n_h==3):
        n_h = 2
        
       # рандомно выбераем из страты нужное кол-во участников в выборку
    if(n_h > 0):       
        df_strat = df_strat.sample(n=n_h) 
    else:
        df_strat = pd.DataFrame()

        
    return df_strat
 

In [18]:
"""
Входные параметры скрипта:
gr_A - данные участников по столбцу от одной группы
gr_B - данные участников по столбцу от другой группы
alpha  –  уровень значимости, формат – дробное число, значение берется из входного параметра alpha  скрипта, который вызывает  скрипт ks_test

"""

def t_test(gr_A,gr_B, alpha):
    
    # t-тест с разными дисперсиями модификация Уэлша
    stat, p_value = ttest_ind(gr_A, gr_B, equal_var=False)[:2]  
    #if(p_value<alpha):
        #print("t-test, НЕ однородны: р=",ks_test)    
    #else:
        #print("t-test, однородны: р=",ks_test) 
    return  p_value 
    

In [19]:
"""
Входные параметры скрипта:
df_aa - данные включающие ключевой столбец идентичный таблице df и фичи для проведения АА-теста, формат – таблица;
df – данные включающие ключевой столбец и фичи с признаками однородности, формат – таблица;
part - количество групп, формат – целые числа, в MVP0 значение всегда 2.
p_key – название ключевого столбца, формат - строковый
alpha – уровень значимости, формат – дробное число, по умолчанию 0.05
representative – флаг репрезентативного отбора, если True, то репрезентативный отбор производится.

"""

def aa_test(df_aa, df, p_key:str ,part:int, alpha:float, representative:bool):
    
    #print('Колонки датасета АА -аа',df_aa.columns)
    
    df = df[['group_num']]
    df_aa = df_aa.sort_values(by = p_key, inplace=False)  
    df_aa = df_aa.astype({p_key: "str"})
    df_aa = df_aa.set_index(p_key)
        
    # Если есть репрезентативный отбор
    if(representative == True):
   
        df_aa=df_aa[df_aa.index.isin(df.index)] 
    #
    df_aa["group_num"] = df["group_num"].values
    
    # чистим данные
    df_aa = sorting_features(df_aa, p_key, True)
    
    
    # проверка однородности      
    flag_aa, df_report_test_aa = checking_uniformity(df_aa, p_group = "group_num", part = part, alpha = alpha)
   
    #print('df',"\n", df.head())
    #print("\n")
    #print('df_aa',"\n",df_aa.head())
    #print("\n")  
    if(len(df) == len(df_aa)):
        
        return flag_aa, df_report_test_aa, df_aa
    

# Код генетического алгоритма, описанный в разделе 1.5.1 отчета о НИР

Схема генетического алгоритма для задачи стартификации:

- Целевая функция - стратифицированная дисперсия 
  $$
    Var[\hat{Y}^{(strat)}] = \frac{1}{N^2}\sum_{h=1}^{H} N_h^2 \left(1 - \frac{n_h}{N_h}\right)\frac{S_h^2}{n_h}\,. \qquad (1)
  $$

- Получить разбиение на атомарные страты, которые впоследствии не бьются на более мелкие
- Случайным образом создать несколько начальных стратификаций над атомарными стартами (начальная популяция)
- Повторять:
  - Найти оптимальные пропорции для сэмплирвоания из каждой страты для каждой стратификации по формуле 
    $$
      n_i = \dfrac{S_i^2 N_i n}{\sum\limits_{j=1}^H S^2_j N_j}\,.\qquad (2)
    $$
  - Посчитать целевую фуцнкцию для каждой стратификации в популяции
  - Породить новую популяцию стратификаций, путем скрещивания особей старого поколения, с маленькими значениями целевой функции
  - К каждой стратификации применить процесс мутации

In [20]:
#применяем к подвыборке исходной ГС размером 5 млн наблюдений
NORM_CONST = 5_000_000
df_inp = df[['ID'] + strat_features].sample(NORM_CONST, random_state=3)
df_aa = df.loc[df_inp.index, ['ID', 'visit']]

In [21]:
%%time
#минимальное число элементов страты для корректного подсчета выборочной дисперсии
min_sample_in_group = 2
#стартовое разбиение на атомарные страты. Может осуществляться любым алгоритмом, за основу взят скрипт, переданный Заказчиком
atomic_strats = get_base_stratification(df_inp, df_aa, p_key="ID", part=min_sample_in_group)  

CPU times: user 1min 51s, sys: 7.18 s, total: 1min 58s
Wall time: 1min 58s


In [22]:
def sample_strata_sizes_(df, strata_dict, p_key, target_df, n):
    """
    Функция возвращает оптимальные пропорции для сэмплирования из страт по формуле (2)
    
    Параметры:
        df - DataFrame - все объекты
        strata_dict - dict[list] - значения в словаре -- индексы объектов, принадлежащих каждой страте
        p_key - str - название колонки, содержащей уникальный идентификатор каждого пользователя
        target_df - DataFrame - значение целевой метрики для всех объектов
        n - int - число объектов для сэмплирования для одной группы
    Выход:
        n_h - array - оптимальные пропорции для каждой страты
    """
    stds = []
    sizes = []
    
    for i in strata_dict:
        df_strat = df[df[p_key].astype(int).isin(strata_dict[i])]
        y = target_df.loc[target_df[p_key].isin(df_strat[p_key]), 'visit']
        stds.append(y.values.std())
        sizes.append(y.shape[0])

    sizes =  np.array(sizes)
    stds = np.array(stds)
    n_h = (sizes * stds * n) / np.sum(stds * sizes)

    return n_h


In [23]:
# Подсчитываем стратифицированную дисперсию по формуле (1) при простом разбиении на страты и сэмплировании с учетом дисперсии
# Выход: V_min - float - стратифицированная дисперсия

n = 10_000 

n_h = sample_strata_sizes_(df_inp, atomic_strats, "ID", df_aa, n)
n_h = np.round(n_h).astype(int)

df_res_all_strat_std = pd.DataFrame()
j = 0

stratified_var_ = 0
for i in tqdm(atomic_strats, position=0, leave=True):
    df_strat = df[df["ID"].astype(int).isin(atomic_strats[i])]
    N = df_strat.shape[0]
    
    var = df_aa.loc[df_aa.ID.isin(df_strat.ID), "visit"].values.var(ddof=1)
    if n_h[j] > 0:
        stratified_var_ += N ** 2 * (1 - n_h[j] / N) * ( var / n_h[j])

    j += 1

V_min = stratified_var_
V_min

  0%|          | 0/89 [00:00<?, ?it/s]

51718864.00642117

In [24]:
atomic_strats_keys = list(atomic_strats.keys())
atomic_strats_ind = list(range(len(atomic_strats_keys)))
atomic_strata_by_ind = dict(zip(atomic_strats_ind, list(atomic_strats.values())))

In [25]:
# Подсчитываем стратифицированное среднее и стратифицированную дисперсию для атомарных страт
# Выход:
#    var_dict - dict - ключ -- номер атомарной страты, значение -- стратифицированная дисперсия атомарной страты
#    mean_dict - dict - ключ -- номер атомарной страты, значение -- стратифицированное среднее атомарной страты

p_key = 'ID'
target_column = 'visit'

variances = []
means = []

for key in atomic_strats_keys:
    data = df[df[p_key].astype(int).isin(atomic_strats[key])]
    if len(data) <= 1:
        variances.append(0.0)
    else:
        variances.append(data[target_column].values.var(ddof=1))
    means.append(data[target_column].values.mean())

var_dict = dict(zip(atomic_strats_ind, variances))
mean_dict = dict(zip(atomic_strats_ind, means))

In [26]:
# Вспомогательные функции для генетического алгоритма
P_KEY = 'ID'
TARGET_COLUMN = 'visit'
TOTAL_SIZE = df_inp.shape[0]


def join_vars_sizes(atomic_strats_numbers):
    """
    Подсчет дисперсии в новом объединении атомарных страт

    Параметры:
        atomic_strats_numbers - list - номера атомарных страт, входящих в данную страту
    Выход:
        var - float - дисперсия страты
        total_size - int - число объектов в страте
    """
    new_mean = join_means(atomic_strats_numbers)
    var = 0
    total_size = 0
    for i in atomic_strats_numbers:
        size = len(atomic_strata_by_ind[i])
        total_size += size


        var += (size - 1) * var_dict[i] + size * (mean_dict[i] - new_mean) ** 2

    if total_size <= 1:
        return 0.0, total_size
    
    var = var / (total_size - 1)
    return var, total_size


def join_means(atomic_strats_numbers):
    """
    Подсчет среднего в новом объединении атомарных страт
    
    Параметры:
        atomic_strats_numbers - list - номера атомарных страт, входящих в данную
    Выход:
        mean - float - среднее в страте
    """
    mean = 0 
    total_size = 0
    for i in atomic_strats_numbers:
        size = len(atomic_strata_by_ind[i])
        total_size += size
        mean += (mean_dict[i] * size)
    
    mean = mean / total_size
    return mean


def minimize_var_solution(all_strats, n):
    """
    Подсчет оптимальных пропорций при сэмплировании из страт
    
    Параметры:
        all_strats - list[list] - лист с элементами (номерами атомарных страт), задающий новую стратификацию
        n - int - число объектов для сэмплирвоания в группу
        
    Выход:
        n_h - array[int] - оптимальное число для сэмплирвоания из каждой страты, посчитанное по формуле (2)
    """
    stds = []
    sizes = []
    for strata in all_strats:
        if len(strata) == 0:
            stds.append(0)
            sizes.append(0)
            continue
        var, size = join_vars_sizes(strata)
        stds.append(np.sqrt(var))
        sizes.append(size)
        
    if VERBOSE:
        print(f"stds: {stds}\n")
    sizes =  np.array(sizes)
    stds = np.array(stds)  
    n_h = (sizes * stds * n) / np.sum(stds * sizes)

    return np.round(n_h).astype(int)

def get_users(atomic_strats_numbers):
    users_indexes = []
    for i in atomic_strats_numbers:
        indexes = atomic_strata_by_ind[i]
        users_indexes.extend(atomic_strata_by_ind[i])
    return users_indexes


def calc_stratified_var(stratification, n_h):
    """
    Считаем стратифицированную дисперсию для разбиения с учетом насэмплированных элементов по формуле (1)
    
    Параметры:
        stratification - list[list] - лист с элементами (номерами атомарных страт) задающий новую стратификацию
        n_h - np.array - оптимальные пропорции для сэмплирования из страт
    
    Выход:
        stratified_var - float - стратифицирванная дисперсия
    """
    stratified_var = 0
    j = 0
    for strat in stratification:
        
        if VERBOSE:
            print(f'starta: {strat}\n')
            
        if len(strat) == 0:
            stratified_var += 0
        
        else:
            var, N = join_vars_sizes(strat)
            
            if N > 1 and n_h[j] > 0:
                stratified_var += N ** 2 * (1 - n_h[j] / N) * ( var / n_h[j])
            else:
                stratified_var += 0

        if VERBOSE:
            print(f"stratified_var: {stratified_var}\n")
        j += 1
        
    return stratified_var



def create_stratification(generation_atom_dict1, generation_atom_dict2, mutation_rate):
    """
    По двум родителям порождаем новую стратификацию
    Параметры:
        generation_atom_dict1 - dict - словарь первого родителя, где ключ - номер атомарной страты, значение - номер страты в новом разбиении
        generation_atom_dict2 - dict - словарь второго родителя, где ключ - номер атомарной страты, значение - номер страты в новом разбиении
        mutation_rate - float - вероятность мутации
    Выход:
        stratification - list[list] - стратификация, каждая страта - лист с элементами (номерами атомарных страт), входящих в нее
        atom_dict - dict - словарь, в котором каждой атомарной страте приписано, к какой страте (в текущем разбиении) она принадлежит 
    """
    C = np.random.randint(1, K) 
    if VERBOSE:
        print(f"C: {C}\n")

    # страты с индексами [0,1,.., C-1] размечаем как в первом родителе
    # страты с индексами [C, C+1, .., K-1] размечаем как во втором родителе


    atom_dict = {}
    stratification = [[] for i in range(U)]
    for i in range(0, K):
       
        mutation_number = np.random.rand(1)[0]
        if mutation_number < mutation_rate:
            # Мутация. С вероятностью mutation_rate номер страты меняется на случайный
            new_strata_index = np.random.randint(0, U)
        elif i < C:
            new_strata_index = generation_atom_dict1[i] 
        else:
            new_strata_index = generation_atom_dict2[i] 

        stratification[new_strata_index].append(i)
        atom_dict[i] = new_strata_index
        if VERBOSE:
            print(f"atom: {i}, mutation_number: {mutation_number}, new_strata_index: {new_strata_index}")
    
    if VERBOSE:
        print(f"stratification: {stratification}\n")
    return stratification, atom_dict



def produce_next_generation(generation, generation_atom_dict, elitism=0.2, mutation_rate=0.0, n=10):
    """
    Основная функция для порождения нового поколения
    Параметры:
        generation - list[list[list]] - лист стратификаций. Каждая стратификация - list[list] с элементами (номерами атомарных страт), входящих в данную страту
        generation_atom_dict - list[dict] - словарь для каждой стратификации в поколении, в котором каждой атомарной страте приписано, к какой страте (в текущем разбиении) она принадлежит
    Выход:
        next_generation - list[list[list]] - лист с новой стратификацией. Каждая стратификация - list[list] с элементами (номерами атомарных страт), входящих в данную страту
        next_generation_atom_dict - list[dict] - словарь для каждой новой стратификации, в котором каждой атомарной страте приписано, к какой страте она принадлежит 
    """
    global BEST_STRATS_WITH_VAR
    n_individuals = len(generation)
    next_generation_atom_dict = []
    next_generation = []

    if VERBOSE:
        print("="*10, 'counting stratified vars', "="*10, '\n')

    stratified_vars = []

    # Подсчет стратифицированной дисперсии для каждой из стратификаций в новом поколении
    for individual in generation:
        if VERBOSE:
            print(f"\n\nindividual: {individual}\n")
        n_h = minimize_var_solution(individual, n)
        if VERBOSE:
            print(f"n_h: {n_h}\n")

        strat_var = calc_stratified_var(individual, n_h)
        if VERBOSE:
            print(f"total strat_var: {strat_var}\n")

        stratified_vars.append(strat_var)

    if VERBOSE:
        print(f"stratified_vars: {stratified_vars}\n")
    #нормировка
    stratified_vars = np.array(stratified_vars) / NORM_CONST

    #сохранение лучшей из текущих стратификаций
    if len(stratified_vars[stratified_vars > 0]) > 0:
        masked_stratified_vars = np.ma.masked_equal(stratified_vars, 0, copy=False)

        if BEST_STRATS_WITH_VAR[1] > np.min(masked_stratified_vars):
            BEST_STRATS_WITH_VAR = (generation[np.argmin(masked_stratified_vars)], np.min(masked_stratified_vars))
            joblib.dump(BEST_STRATS_WITH_VAR, 'best_strat_and_var_criteo.joblib')
    #вероятность выживания стратификации связана с оценкой стратифицированной дисперсии соответствующего разбиения
    probs = scipy.special.softmax(-stratified_vars)

    if VERBOSE:
        print(f"probs: {probs}\n\n\n\n")


    if VERBOSE:
        print("="*10, 'choosing best stratifications', "="*10, '\n')

    # Отбор лучших стратификаций в новое поколение
    part = np.round(elitism * len(generation)).astype(int)
    indexes_of_best = probs.argsort()[-part:]
    if VERBOSE:
        print(f"part: {part}, indexes_of_best: {indexes_of_best}\n")

    
    if VERBOSE:
        print("="*10, 'creating new generation', "="*10, '\n')
    for i in range(part):
        next_generation.append(generation[indexes_of_best[i]])
        next_generation_atom_dict.append(generation_atom_dict[indexes_of_best[i]])

    if VERBOSE:
        print(f"next_generation_best: {next_generation}, next_generation_best_dict: {next_generation_atom_dict}\n")

    
    # Генрация нового поколения путем скрещивания и мутаций
    n_individuals = n_individuals - part

    for i in range(0, n_individuals):
        if VERBOSE:
            print(f"\n\n\nindividual: {i}\n")

        # Сэмплирование двух стратификации с вероятностями, пропорциональными стратифицированной дисперсии 
        indexes = np.arange(0, len(generation))
        parents_indexes = np.random.choice(indexes, size=2, replace=False, p=probs)
        #выбор особей для скрещивания
        parent1 = generation_atom_dict[parents_indexes[0]]
        parent2 = generation_atom_dict[parents_indexes[1]]

        if VERBOSE:
            print(f"chosen parents: {parents_indexes}, dict1: {parent1}, dict2: {parent2}\n")
        #порождение потомка по родителям, выбранным выше
        child, child_atom_dict = create_stratification(parent1, parent2, mutation_rate)

        if VERBOSE:
            print(f"child: {child}, child_atom_dict: {child_atom_dict}\n")
        #сохранение ребенка в новом поколении
        next_generation.append(child)
        next_generation_atom_dict.append(child_atom_dict)
    
    if VERBOSE:
        print(next_generation)
    if VERBOSE:
        print(next_generation_atom_dict)
    
    return next_generation, next_generation_atom_dict

In [27]:
def create_atom_dicts(generation):
    """
    Создание словаря для стратификаций. Каждой атомарной страте сопоставляется номер новой страты, которой она принадлежит.

    Параметры:
        generation - list[list[list]] - лист стратификаций. Каждая стратификация - list[list] с элементами (номерами атомарных страт), входящих в данную страту
    Выход:
        dicts - list[dict] - лист со словарями для каждой стратификации, где каждой атомарной страте приписано, к какой страте она принадлежит 
    """
    dicts = []
    for stratification in generation:
        curr_dict = {}
        for start_ind in range(len(stratification)):
            strat = stratification[start_ind]
            for i in strat:
                curr_dict[i] = start_ind
        
        dicts.append(curr_dict)

    return dicts

In [28]:
def produce_init_generation(num_individual, num_atomic_strats, U, method='rand'):
    """
    Порождает изначальные стратификации случайным образом. Возвращает изначальные num_individual стратификаций над атомарными.

    Параметры:
        num_individual - int - число изначальных стратификаций
        method - str - каким методом создавать эти стратификации 
        num_atomic_strats - int - число атомарных страт
        U - int - максимальное чичло страт в каждой стартификации
    Выход:
        generation - list[list[list]] - лист стратификаций. Каждая стратификация - list[list] с элементами (номерами атомарных страт), входящих в данную страту
        atom_dicts - list[dict] - словарь для каждой стратификации в поколении, в котором каждой атомарной страте приписано, к какой страте (в текущем разбиении) она принадлежит
    """
    generation = []

    if method == 'rand':
        if num_atomic_strats == U:
            num_individual = num_individual - 1
            
        for i in range(num_individual):
            individual = [[] for _ in range(U)]
            strat_vector = np.random.randint(0, U, num_atomic_strats)
            for j in range(num_atomic_strats):
                index_strat = strat_vector[j]
                individual[index_strat].append(j)
            generation.append(individual)
    
    else:
        raise ValueError('Метод не имплементирован')
    


    atom_dicts = create_atom_dicts(generation)

    if num_atomic_strats == U:
        init_startification = np.arange(0, num_atomic_strats, 1).reshape((-1, 1))
        init_dict = dict(zip(np.arange(0, num_atomic_strats, 1), np.arange(0, num_atomic_strats, 1)))

        generation.append(init_startification.tolist())
        atom_dicts.append(init_dict)

    return generation, atom_dicts

In [29]:
## Для создания файла для логирования нужно раскомментировать следующий код

# !rm -rf logs_criteo.txt
# !touch logs_criteo.txt
# !cat logs_criteo.txt

In [30]:
%%time

# Запуск генетического алгоритма
# данный параметр можно увеличивать, для гарантированного воспроизведения результатов отчета поставить
#max_iter = 3_000_000
max_iter = 30_000      # Максимальное число итераций
V_limit = V_min         # Пороговое знаечение для стратифицированной дисперсии, если curr_var < V_limit => выходим
mutation_rate = 0.001   # Вероятность мутировать (выбрать рандомную метку для объекта)
n = 10_000              # Размер выборки из генеральной совокупности
elitism = 0.3           # Доля лучших решений, не мутирующая при переходе в следующее поколение
n_init = 20             # Число особей, поддерживаемое на каждом шаге генетического алгоритма


K = len(atomic_strata_by_ind.keys())
U = K
BEST_STRATS_WITH_VAR = (([], np.inf))

input_ = produce_init_generation(n_init, K, U, method='rand') 


VERBOSE = False

iter_num = 0
logs_file = open("logs_criteo.txt","a")

logs_file.write(f"Init var: {V_min / NORM_CONST}, mut_rate: {mutation_rate}\n\n\n")
logs_file.flush()

for iter_num in tqdm(range(0, max_iter)):

    input_ = produce_next_generation(input_[0], input_[1], elitism=elitism, mutation_rate=mutation_rate, n=n)
    
    logs_file.write(f"Iter number: {iter_num} | Best varance: {BEST_STRATS_WITH_VAR[1]}\n\n")
    logs_file.flush()
    iter_num += 1

logs_file.write(f"THE END\n")
logs_file.close()

  0%|          | 0/30000 [00:00<?, ?it/s]

CPU times: user 3min 43s, sys: 5.58 s, total: 3min 48s
Wall time: 3min 46s


In [31]:
BEST_STRATS_WITH_VAR = joblib.load("best_strat_and_var_criteo.joblib")

In [32]:
non_empty_strats = 0
for i in BEST_STRATS_WITH_VAR[0]:
    non_empty_strats += 1 if len(i) > 0 else 0

In [33]:
data = [[round(V_min / NORM_CONST, 5), len(atomic_strats_ind)], [round(BEST_STRATS_WITH_VAR[1], 5), non_empty_strats]]

compar_table = pd.DataFrame(data, index=['Original stratification', 'GA stratification'], columns=['Var', "# Strat"])

compar_table

,Var,# Strat
Original stratification,10.34377,89
GA stratification,10.33104,56
